In this exercise, you'll redo the data gathering phase of the UNData Exploration project by using APIs instead of downloading csv files.

You'll make use of the [World Bank Indicators API](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation). Note that this API does not require an API key. Before attempting the exercise, it would be a good idea to skim through the Documentation page and to check out the [Basic Call Structure article](https://datahelpdesk.worldbank.org/knowledgebase/articles/898581).

In [38]:
import requests
import pandas as pd

1. Use the API to get all available data for the _GDP per capita, PPP (constant 2017 international $)_ indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [77]:
endpoint = 'http://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD'

params = {
    'format': 'json',
    'per_page': '20000'
}

response = requests.get(endpoint, params = params)

response.status_code

200

In [78]:
# note: I could start with default poer_pgae and take response.json()[0]['total']
# and then go back and adjust the total number of responses
response.json()[0]

{'page': 1,
 'pages': 1,
 'per_page': 20000,
 'total': 16758,
 'sourceid': '2',
 'lastupdated': '2023-09-19'}

In [41]:
gdp_df = pd.json_normalize(response.json()[1])

In [42]:
gdp_df = (
    gdp_df
    .rename(columns={
        'date': 'Year',
        'value': 'GDP_per_capita',
        'country.value': 'Country'})
    .drop(columns=[
        'countryiso3code',
        'unit',
        'obs_status',
        'decimal',
        'indicator.id',
        'indicator.value',
        'country.id'])
)

gdp_df.head()

,Year,GDP_per_capita,Country
0,2022,3553.913370,Africa Eastern and Southern
1,2021,3519.174840,Africa Eastern and Southern
2,2020,3455.023119,Africa Eastern and Southern
3,2019,3648.220302,Africa Eastern and Southern
4,2018,3661.360566,Africa Eastern and Southern


2. Now, use the API to get all available data for _Life expectancy at birth, total (years)_. This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [43]:
endpoint = 'http://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN'

params = {
    'format': 'json',
    'per_page': '20000'
}

response = requests.get(endpoint, params = params)

response.status_code

200

In [44]:
response.json()[0]

{'page': 1,
 'pages': 1,
 'per_page': 20000,
 'total': 16758,
 'sourceid': '2',
 'lastupdated': '2023-09-19'}

In [45]:
le_df = pd.json_normalize(response.json()[1])

In [46]:
le_df = (
    le_df
    .rename(columns={
        'countryiso3code': 'ISO',
        'date': 'Year',
        'value': 'Life_Expectancy',
        'country.value': 'Country'})
    .drop(columns=[
        'unit',
        'obs_status',
        'decimal',
        'indicator.id',
        'indicator.value',
        'country.id'])
)

le_df.head()

,ISO,Year,Life_Expectancy,Country
0,AFE,2022,NaN,Africa Eastern and Southern
1,AFE,2021,62.454590,Africa Eastern and Southern
2,AFE,2020,63.313860,Africa Eastern and Southern
3,AFE,2019,63.755678,Africa Eastern and Southern
4,AFE,2018,63.365863,Africa Eastern and Southern


In [47]:
(
    le_df
    .set_index(le_df['ISO'] + '-' + le_df['Year'])
    .drop(columns=['ISO', 'Year'])
    .head()
)

,Life_Expectancy,Country
AFE-2022,NaN,Africa Eastern and Southern
AFE-2021,62.454590,Africa Eastern and Southern
AFE-2020,63.313860,Africa Eastern and Southern
AFE-2019,63.755678,Africa Eastern and Southern
AFE-2018,63.365863,Africa Eastern and Southern


In [49]:
print(len(gdp_df))
print(len(le_df))

16758
16758


3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.


In [51]:
gdp_le_df = gdp_df.merge(le_df, how='inner', on=['Year', 'Country'])
gdp_le_df.head()

,Year,GDP_per_capita,Country,ISO,Life_Expectancy
0,2022,3553.913370,Africa Eastern and Southern,AFE,NaN
1,2021,3519.174840,Africa Eastern and Southern,AFE,62.454590
2,2020,3455.023119,Africa Eastern and Southern,AFE,63.313860
3,2019,3648.220302,Africa Eastern and Southern,AFE,63.755678
4,2018,3661.360566,Africa Eastern and Southern,AFE,63.365863


4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the [Country API](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries). Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [80]:
endpoint = 'http://api.worldbank.org/v2/country/'
params = {
    #'indicator': 'NY.GDP.PCAP.PP.KD',
    'format': 'json',
    'per_page': '20000'
    #'page': '3:5'
}

response = requests.get(endpoint, params = params)

response.status_code

200

In [81]:
response.json()[0]

{'page': 1, 'pages': 1, 'per_page': '20000', 'total': 297}

In [82]:
countries = pd.json_normalize(response.json()[1])

In [85]:
countries = (
    countries
    .loc[countries['region.value'] != 'Aggregates']
    .rename(columns={
        'name': 'Country',
        'region.value': 'Region'})
    [['Country', 'Region', 'capitalCity']]
)

countries.head()

,Country,Region,capitalCity
0,Aruba,Latin America & Caribbean,Oranjestad
2,Afghanistan,South Asia,Kabul
5,Angola,Sub-Saharan Africa,Luanda
6,Albania,Europe & Central Asia,Tirane
7,Andorra,Europe & Central Asia,Andorra la Vella


In [86]:
print(gdp_le_df.shape[0])
print(gdp_le_df['Country'].nunique())
print(countries.shape[0])

16758
266
218


In [87]:
gdp_le_final = gdp_le_df.merge(countries, how='inner', on='Country')

In [88]:
print(gdp_le_final['Country'].nunique())
print('countries that are missing:', set(countries['Country'].unique()) - set(gdp_le_final['Country'].unique()))
print(gdp_le_final.shape[0])

217
countries that are missing: {'Taiwan, China'}
13671


In [91]:
# countries with no capital (and therefore no longitude and latitude)
gdp_le_final.loc[gdp_le_final['capitalCity'] == '', 'Country'].unique()

array(['Channel Islands', 'Gibraltar', 'Hong Kong SAR, China', 'Israel',
       'Macao SAR, China', 'West Bank and Gaza'], dtype=object)

In [29]:
#print(set(gdp_le_df['Country'].unique()) - set(gdp_le_final['Country'].unique()))

In [27]:
gdp_le_df.head()

,Year,GDP_per_capita,Country,ISO,Life_Expectancy
0,2022,3553.913370,Africa Eastern and Southern,AFE,NaN
1,2021,3519.174840,Africa Eastern and Southern,AFE,62.454590
2,2020,3455.023119,Africa Eastern and Southern,AFE,63.313860
3,2019,3648.220302,Africa Eastern and Southern,AFE,63.755678
4,2018,3661.360566,Africa Eastern and Southern,AFE,63.365863


In [64]:
print(gdp_le_df.loc[gdp_le_df['Country'] == 'Canada', 'ISO'].unique())
print(gdp_le_df.loc[gdp_le_df['Country'] == 'United States', 'ISO'].unique())

['CAN']
['USA']


**Bonus Questions:** After doing a basic request to get all records, you can attempt these exercises to get additional practice using query parameters.

1. Adjust your request so that it returns data just for the United States.

2. Adjust your request so that it returns data just for the United States for the year 2021.

3. Adjust your request so that it returns data just for the United States for the years 2000 through 2021.

4. Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.

In [71]:
endpoint = 'http://api.worldbank.org/v2/country/USA;CAN/indicator/NY.GDP.PCAP.PP.KD'

params = {
    'format': 'json',
    'per_page': '20000',
    'date':'2000:2021'
}

response = requests.get(endpoint, params = params)

response.status_code

200

In [72]:
response.json()[0]

{'page': 1,
 'pages': 1,
 'per_page': 20000,
 'total': 44,
 'sourceid': '2',
 'lastupdated': '2023-09-19'}

In [73]:
response.json()[1]

[{'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
   'value': 'GDP per capita, PPP (constant 2017 international $)'},
  'country': {'id': 'CA', 'value': 'Canada'},
  'countryiso3code': 'CAN',
  'date': '2021',
  'value': 48218.038315768,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
   'value': 'GDP per capita, PPP (constant 2017 international $)'},
  'country': {'id': 'CA', 'value': 'Canada'},
  'countryiso3code': 'CAN',
  'date': '2020',
  'value': 46181.7575545756,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
   'value': 'GDP per capita, PPP (constant 2017 international $)'},
  'country': {'id': 'CA', 'value': 'Canada'},
  'countryiso3code': 'CAN',
  'date': '2019',
  'value': 49175.6770500696,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
   'value': 'GDP per capita, PPP (constant 2017 international $)'},
  'country': {'id': 'CA', 'value': 'Canad

5. If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.

6. The endpoint "http://api.worldbank.org/v2/indicator" will return information about all available indicators. See [Indicator API Queries](https://datahelpdesk.worldbank.org/knowledgebase/articles/898599-indicator-api-queries). Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator.